In [ ]:
import random
import glob
import pathlib
import os
from PIL import Image
import numpy as np
import sys
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import gc
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
label_to_index = {'Normal': 0, 'People': 1, 'Potential Pileup': 2, 'Pileup': 3}
index_to_label = {v: k for k, v in label_to_index.items()}

In [ ]:
IMG_SIZE = 224

def preprocess_image(path):
    image = Image.open(path)
    return np.array((image.resize((IMG_SIZE, IMG_SIZE)))) / 255.0

In [ ]:
#Load test data from the test folder into x_test, y_test
def getTestImages(path):
    image_paths = glob.glob(path)
    random.shuffle(image_paths)
    return image_paths

image_paths = getTestImages("CompactorTest\\Normal\\*.jpg")
y_test = [0] * len(image_paths)
image_paths += getTestImages("CompactorTest\\People\\*.jpg")
y_test += [1] * (len(image_paths) - len(y_test))
image_paths += getTestImages("CompactorTest\\PotentialPileup\\*.jpg")
y_test += [2] * (len(image_paths) - len(y_test))
image_paths += getTestImages("CompactorTest\\Pileup\\*.jpg")
y_test += [3] * (len(image_paths) - len(y_test))

x_test = np.zeros(shape=(len(image_paths), IMG_SIZE, IMG_SIZE, 3))
for i, image in tqdm.tqdm(enumerate(image_paths)):
    x_test[i] = preprocess_image(image)

In [ ]:
model = keras.models.load_model('GarbageModel.h5')

In [ ]:
predictions = model.predict(x_test)

In [ ]:
#check false positives
fp = np.zeros(16).reshape((4,4))
paths = []
for i in range(len(fp)):
    paths.append([])
    for j in range(len(fp[i])):
        paths[i].append([])
correct = np.zeros(4)
totals = np.zeros(4)
for i, prediction in enumerate(predictions):
    correctLabel = y_test[i]
    predictedLabel = np.argmax(prediction)
    totals[correctLabel] += 1
    if(predictedLabel != correctLabel): #when it is incorrect
        fp[correctLabel][predictedLabel] += 1
        paths[correctLabel][predictedLabel].append(image_paths[i])
    else:
        correct[correctLabel] += 1
print("False Positives: ")
for i in range(len(fp)):
    print(index_to_label[i], fp[i])
print('\nCorrect: ')
for i in range(len(correct)):
    print(index_to_label[i], correct[i], '/', totals[i])
#print(paths[2][0]) #line to print image paths to see what images are the false positives for the given class

In [ ]:
image_path = "CompactorLabeled\\Pileup\\compactor1_03231.jpg"
image = Image.open(image_path)
image = np.array((image.resize((IMG_SIZE, IMG_SIZE)))) / 255.0
image = np.reshape(image, [1,IMG_SIZE, IMG_SIZE, 3])
classes = model.predict_classes(image)
print(index_to_label[classes[0]])
plt.imshow(np.reshape(image, [IMG_SIZE, IMG_SIZE, 3]))